<a href="https://colab.research.google.com/github/mohripan/Machine-Learning/blob/main/Pytorch_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [26]:
class NN(nn.Module):
  def __init__(self, input_size, output_size):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 128)
    self.fc2 = nn.Linear(128, 128)
    self.fc3 = nn.Linear(128, output_size)

  def forward(self, X):
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)
    return X

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [28]:
input_size = 784
output_size = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

In [29]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [30]:
model = NN(input_size, output_size).to(device)

In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [32]:
import numpy as np

In [33]:
train_losses = np.zeros(num_epochs)
test_losses = np.zeros(num_epochs)

for i in range(num_epochs):
  train_loss = []
  for batch_idx, (data, targets) in enumerate(train_loader):
    optimizer.zero_grad()

    data, targets = data.to(device), targets.to(device)

    data = data.reshape(data.shape[0], -1)

    scores = model(data)
    loss = criterion(scores, targets)

    train_loss.append(loss.item())

    loss.backward()
    optimizer.step()

  train_loss = np.mean(train_loss)
  train_losses[i] = train_loss

  test_loss = []

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():

    for batch_idx, (data, targets) in enumerate(test_loader):
      data, targets = data.to(device), targets.to(device)

      data = data.reshape(data.shape[0], -1)

      scores = model(data)
      loss = criterion(scores, targets)

      _, predictions = scores.max(1)
      num_correct += (predictions == targets).sum()
      num_samples += predictions.size(0)

      test_loss.append(loss.item())

      acc = float(num_correct)/float(num_samples)*100

  test_loss = np.mean(test_loss)
  test_losses[i] = test_loss

  print(f'Epochs: {i+1}/{num_epochs}, train_loss: {train_loss}, test_loss: {test_loss}, Test accuracy: {acc}')

Epochs: 1/10, train_loss: 0.31280439868887094, test_loss: 0.15459419945610842, Test accuracy: 95.37
Epochs: 2/10, train_loss: 0.12707158334668417, test_loss: 0.10971448570341584, Test accuracy: 96.67
Epochs: 3/10, train_loss: 0.0874579904073758, test_loss: 0.08824128396571822, Test accuracy: 97.2
Epochs: 4/10, train_loss: 0.06456104432047009, test_loss: 0.08142106495237802, Test accuracy: 97.32
Epochs: 5/10, train_loss: 0.05036966475715954, test_loss: 0.08214104594292036, Test accuracy: 97.47
Epochs: 6/10, train_loss: 0.041267792734544353, test_loss: 0.08631752496320164, Test accuracy: 97.43
Epochs: 7/10, train_loss: 0.033110116973634994, test_loss: 0.07545758044385355, Test accuracy: 97.65
Epochs: 8/10, train_loss: 0.027189154253425156, test_loss: 0.08273697063674598, Test accuracy: 97.72999999999999
Epochs: 9/10, train_loss: 0.022530051832221253, test_loss: 0.09087039864909897, Test accuracy: 97.61999999999999
Epochs: 10/10, train_loss: 0.020657563840118753, test_loss: 0.087795480732

In [34]:
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for X, y in loader:
      X, y = X.to(device), y.to(device)

      X = X.reshape(X.shape[0], -1)

      scores = model(X)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    acc = float(num_correct)/float(num_samples)*100
    print(f'Got {num_correct}/{num_samples} with accuracy: {acc}')

In [35]:
check_accuracy(test_loader, model)

Got 9774/10000 with accuracy: 97.74000000000001
